<a href="https://colab.research.google.com/github/AmishaSingh0408/texttosqlapp/blob/main/d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google import genai
import psycopg2



### CONFIG


In [ ]:
GEMINI_API_KEY = ""

DB_CONFIG = {
    "host": "localhost",
    "database": "postgres",
    "user": "postgres",
    "password": "",
    "port":
}


---


In [ ]:

# Setup Gemini
client = genai.Client(api_key=GEMINI_API_KEY)

def get_schema(conn):
    cursor = conn.cursor()
    cursor.execute("""
        SELECT table_name, column_name, data_type
        FROM information_schema.columns
        WHERE table_schema = 'public'
        ORDER BY table_name, ordinal_position;


)
    rows = cursor.fetchall()
    
    schema = {}
    for table, column, dtype in rows:
        if table not in schema:
            schema[table] = []
        schema[table].append(f"{column} ({dtype})")
    
    schema_text = ""
    for table, cols in schema.items():
        schema_text += f"Table: {table}\n  Columns: {', '.join(cols)}\n\n"
    
    return schema_text

def text_to_sql(question, schema):
    prompt = f


In [ ]:
You are a SQL expert. Given the database schema below, convert the user's question into a valid PostgreSQL query.
Return ONLY the SQL query, nothing else. No explanation, no markdown, no backticks.

Schema:
{schema}

Question: {question}

SQL Query:



    response = client.models.generate_content(
        model="gemini-1.5-flash",
        contents=prompt
    )
    return response.text.strip()

def run_query(conn, sql):
    cursor = conn.cursor()
    cursor.execute(sql)
    rows = cursor.fetchall()
    col_names = [desc[0] for desc in cursor.description]
    return col_names, rows

def main():
    print("Connecting to PostgreSQL...")
    conn = psycopg2.connect(**DB_CONFIG)
    print("Connected!\n")
    
    schema = get_schema(conn)
    print("Schema loaded:\n")
    print(schema)
    
    while True:
        question = input("Ask a question (or type 'exit'): ")
        if question.lower() == 'exit':
            break
        
        print("\nGenerating SQL...")
        sql = text_to_sql(question, schema)
        print(f"\nGenerated SQL:\n{sql}\n")
        
        try:
            cols, rows = run_query(conn, sql)
            print("Results:")
            print(" | ".join(cols))
            print("-" * 60)
            for row in rows:
                print(" | ".join(str(val) for val in row))
        except Exception as e:
            print(f"Error running query: {e}")
        
        print("\n" + "="*60 + "\n")
    
    conn.close()

if __name__ == "__main__":
    main()
